# CartPole-v1

In [1]:
# 1) Imports
import gymnasium as gym
import matplotlib.pyplot as plt

In [2]:
# 2) Create the environment
#    – render_mode='rgb_array' lets you grab frames (e.g. for plotting later)
env = gym.make("CartPole-v1", render_mode="rgb_array")

In [ ]:
# 3) Inspect spaces
print("Observation space:", env.observation_space)  # Box(4,)
print("Action space:", env.action_space)            # Discrete(2)


space = env.observation_space.shape
action = env.action_space
print(space[0])
print(action)

l = action
print(l.n)

action

Observation space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Action space: Discrete(2)
4
Discrete(2)
2


1

In [10]:
import random

print(random.random())


0.5582578445538783


In [4]:
# 4) Reset and take a few random steps
obs, info = env.reset(seed=42)
frames = []
for step in range(10):
    action = env.action_space.sample()              # random action: 0 or 1
    obs, reward, done, truncated, info = env.step(action)
    frame  = env.render()
    print(frame.shape)
    frames.append(env.render())
    print(f"Step {step:02d} – obs={obs.round(2)}, reward={reward}, done={done}")
    if done:
        obs, info = env.reset() 

frames_np = np.stack(frames)  # Shape: (num_frames, alto, ancho, 3)
frames_tensor = torch.from_numpy(frames_np)  # Tensor de shape igual

# Si quieres que el canal de color sea la segunda dimensión (formato PyTorch: N, C, H, W):
frames_tensor = frames_tensor.permute(0, 3, 1, 2)  # (num_frames, 3, alto, ancho)
    
    


c:\Users\maria\miniconda3\envs\dqn-env\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


(400, 600, 3)
Step 00 – obs=[ 0.03  0.19  0.04 -0.26], reward=1.0, done=False
(400, 600, 3)
Step 01 – obs=[ 0.03  0.38  0.03 -0.54], reward=1.0, done=False
(400, 600, 3)
Step 02 – obs=[ 0.04  0.58  0.02 -0.83], reward=1.0, done=False
(400, 600, 3)
Step 03 – obs=[ 0.05  0.77  0.   -1.11], reward=1.0, done=False
(400, 600, 3)
Step 04 – obs=[ 0.07  0.97 -0.02 -1.4 ], reward=1.0, done=False
(400, 600, 3)
Step 05 – obs=[ 0.09  0.77 -0.05 -1.12], reward=1.0, done=False
(400, 600, 3)
Step 06 – obs=[ 0.1   0.97 -0.07 -1.42], reward=1.0, done=False
(400, 600, 3)
Step 07 – obs=[ 0.12  0.77 -0.1  -1.15], reward=1.0, done=False
(400, 600, 3)
Step 08 – obs=[ 0.14  0.97 -0.12 -1.47], reward=1.0, done=False
(400, 600, 3)
Step 09 – obs=[ 0.15  1.17 -0.15 -1.8 ], reward=1.0, done=False


NameError: name 'np' is not defined

In [ ]:
# 5) Close when done
env.close()